In [1]:
from rllab.sampler.utils import rollout
import joblib
import tensorflow as tf

In [6]:
speedup = 1
max_path_length = 1000
file = "../data/local/2-14-dev-linear-ocm--branch-fix-manual-qf/2-14-dev-linear-ocm--branch-fix-manual-qf_2017_02_14_16_55_59_0000--s-0/params.pkl"


In [7]:
tf.reset_default_graph()
with tf.Session() as sess:
    data = joblib.load(file)    

In [9]:
if 'policy' in data:
    policy = data['policy']
else:
    qf = data['optimizable_qfunction']
    policy = qf.implicit_policy
qf = data['eq']
env = data['env']

In [2]:
print("Policy loaded")
while True:
    try:
        path = rollout(env, policy, max_path_length=args.max_path_length,
                       animated=True, speedup=args.speedup)
    # Hack for now. Not sure why rollout assumes that close is an
    # keyword argument
    except TypeError as e:
        if (str(e) != "render() got an unexpected keyword "
                      "argument 'close'"):
            raise e

SyntaxError: invalid syntax (<ipython-input-2-42017174742f>, line 11)

In [11]:
vars(qf)

{'_Serializable__args': ('critic',),
 '_Serializable__kwargs': {'env_spec': <rllab.envs.env_spec.EnvSpec at 0x7fa4000db0b8>},
 '_batch_norm': False,
 '_batch_norm_config': None,
 '_bn_stat_update_ops': [],
 '_cached_assign_ops': {<tensorflow.python.ops.variables.Variable at 0x7fa4000e3898>: <tf.Tensor 'Assign:0' shape=(12, 1) dtype=float32_ref>,
  <tensorflow.python.ops.variables.Variable at 0x7fa4000e3dd8>: <tf.Tensor 'Assign_1:0' shape=(1,) dtype=float32_ref>},
 '_cached_assign_placeholders': {<tensorflow.python.ops.variables.Variable at 0x7fa4000e3898>: <tf.Tensor 'Placeholder:0' shape=<unknown> dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7fa4000e3dd8>: <tf.Tensor 'Placeholder_1:0' shape=<unknown> dtype=float32>},
 '_cached_param_dtypes': {(): [dtype('float32'), dtype('float32')]},
 '_cached_param_shapes': {(): [(12, 1), (1,)]},
 '_cached_params': {(): [<tensorflow.python.ops.variables.Variable at 0x7fa4000e3898>,
   <tensorflow.python.ops.variables.Variable at 